# Spectra

### Imports

In [4]:
import emcee, corner
import numpy as np
import matplotlib.pyplot as plt
from model import read_data, doppler_shift, filter_by_theta, get_spectra, main

% matplotlib inline

In [5]:
from mpl_toolkits.axes_grid1 import ImageGrid

plt.rc('text', usetex=True)
plt.rc('font', size=48)

### Constants

In [6]:
NUM_THETAS = 9
THETAS_RANGE = np.arccos(np.linspace(1, 0, NUM_THETAS))
THETAS_SELECTED = [0, 45, 90]
THETAS_SELECTED_INDS = [0, 2, 7] # normal: 0-1, 2-3, 7-8
THETAS_LINEWIDTHS = [1, 4, 4]
THETAS_ALPHAS = [1, 0.4, 1]

### Functions

In [7]:
def plot_grid_i(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vlab):
    data = read_data(vrot, vout, logtau)
    for t in range(len(THETAS_SELECTED)):

        ts = THETAS_SELECTED[t]
        tsi = THETAS_SELECTED_INDS[t]
        tl = THETAS_LINEWIDTHS[t]
        ta = THETAS_ALPHAS[t]

        data_theta = filter_by_theta(data, THETAS_RANGE[tsi], THETAS_RANGE[tsi+1])
        x, y = get_spectra(data_theta, as_hist=False)

        theta_lab = r'$\theta=$ ' + '${0:.0f}$'.format(ts) + '$^\circ$'

        if i == 2:
            grid_i.hist(x, weights=y, histtype='step', fill=False, normed=True, 
                        color='black', linewidth=tl, bins=50, alpha=ta, label=theta_lab)
            grid_i.legend(loc='upper right')
        else:
            grid_i.hist(x, weights=y, histtype='step', fill=False, normed=True, 
                        color='black', linewidth=tl, bins=50, alpha=ta)

    props = dict(boxstyle='square', facecolor='white')
    logtau_lab = r'$\tau=10$ ' + '$^{0:.0f}$'.format(logtau) 

    if i <= 2:
        grid_i.text((xmin+xmax)/2, ymax*1.2, vlab, bbox=props,
                    horizontalalignment='center', verticalalignment='center')
    if i%3 == 2:
        grid_i.text(xmax*1.2, (ymin+ymax)/2, logtau_lab, bbox=props, rotation=270,
                    horizontalalignment='center', verticalalignment='center')
    if i == 3:
        grid_i.set_ylabel('$\mathrm{Intensity}$ ($\mathrm{Arbitrary}$ $\mathrm{Units}$)')
    if i == 7:
        grid_i.set_xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
        
    grid_i.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
    grid_i.set_xlim(xmin,xmax)
    grid_i.set_ylim(ymin,ymax)

    
def make_grid_plot_voutfixed(logtaus, vrots, vout):
    num_rows, num_cols = len(logtaus), len(vrots)
    fig = plt.figure(1, figsize=(35, 23))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    xmin, xmax = -250,750
    ymin, ymax = 0, 0.012
    
    i = 0 # grid cell number
    for logtau in logtaus:
        for vrot in vrots:
            grid_i = grid[i]
            vrot_lab = r'$v_{\mathrm{rot}}=$ ' + '${0:.0f}$'.format(vrot) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            plot_grid_i(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vrot_lab)
            i += 1

    plt.savefig('./results/varying_rotation.pdf', format='pdf', transparent=False)
    plt.close()
    
    
def make_grid_plot_vrotfixed(logtaus, vrot, vouts):
    num_rows, num_cols = len(logtaus), len(vouts)
    fig = plt.figure(1, figsize=(35, 23))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    xmin, xmax = -300,750
    ymin, ymax = 0, 0.0125
        
    i = 0 # grid cell number
    for logtau in logtaus:
        for vout in vouts:
            grid_i = grid[i]
            vout_lab = r'$v_{\mathrm{out}}=$ ' + '${0:.0f}$'.format(vout) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            plot_grid_i(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vout_lab)
            #grid_i.set_xticks([-200,0,200,400,600], [-200,0,200,400,600])
            i += 1

    plt.savefig('./results/varying_outflow.pdf', format='pdf', transparent=False)#, bbox_inches=4)
    plt.close()

In [8]:
def plot_grid_i_small(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vlab):
    data = read_data(vrot, vout, logtau)
    for t in range(len(THETAS_SELECTED)):

        ts = THETAS_SELECTED[t]
        tsi = THETAS_SELECTED_INDS[t]
        tl = THETAS_LINEWIDTHS[t]
        ta = THETAS_ALPHAS[t]

        data_theta = filter_by_theta(data, THETAS_RANGE[tsi], THETAS_RANGE[tsi+1])
        x, y = get_spectra(data_theta, as_hist=False)

        theta_lab = r'$\theta=$ ' + '${0:.0f}$'.format(ts) + '$^\circ$'

        if i == 1:
            grid_i.hist(x, weights=y, histtype='step', fill=False, normed=True, 
                        color='black', linewidth=tl, bins=50, alpha=ta, label=theta_lab)
            grid_i.legend(loc='upper right')
        else:
            grid_i.hist(x, weights=y, histtype='step', fill=False, normed=True, 
                        color='black', linewidth=tl, bins=50, alpha=ta)

    props = dict(boxstyle='square', facecolor='white')
    logtau_lab = r'$\tau=10$ ' + '$^{0:.0f}$'.format(logtau) 

    if i <= 1:
        grid_i.text((xmin+xmax)/2, ymax*1.2, vlab, bbox=props,
                    horizontalalignment='center', verticalalignment='center')
    if i%2 == 1:
        grid_i.text(xmax*1.2, (ymin+ymax)/2, logtau_lab, bbox=props, rotation=270,
                    horizontalalignment='center', verticalalignment='center')
    else:
        grid_i.set_ylabel('$\mathrm{Intensity}$ ($\mathrm{AU}$)')

    if i >= 2:
        grid_i.set_xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
        
    grid_i.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
    grid_i.set_xlim(xmin,xmax)
    grid_i.set_ylim(ymin,ymax)


def make_grid_plot_voutfixed_small(logtaus, vrots, vout):
    num_rows, num_cols = len(logtaus), len(vrots)
    fig = plt.figure(1, figsize=(30, 20))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    xmin, xmax = -200,400
    ymin, ymax = 0, 0.011
        
    i = 0 # grid cell number
    for logtau in logtaus:
        for vrot in vrots:
            grid_i = grid[i]
            vrot_lab = r'$v_{\mathrm{rot}}=$ ' + '${0:.0f}$'.format(vrot) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            plot_grid_i_small(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vrot_lab)
            i += 1

    plt.savefig('./results/varying_rotation_small.pdf', format='pdf', transparent=False)#, bbox_inches='tight')
    plt.close()
    
    
def make_grid_plot_vrotfixed_small(logtaus, vrot, vouts):
    num_rows, num_cols = len(logtaus), len(vouts)
    fig = plt.figure(1, figsize=(30, 20))
    grid = ImageGrid(fig, 111, # similar to subplot(111)
                     nrows_ncols = (num_rows, num_cols), # creates num_rows x num_cols grid of axes
                     axes_pad=0.3, # pad between axes in inch.
                     aspect=False,)
    xmin, xmax = -200, 400
    ymin, ymax = 0, 0.0125
    
    i = 0 # grid cell number
    for logtau in logtaus:
        for vout in vouts:
            grid_i = grid[i]
            vout_lab = r'$v_{\mathrm{out}}=$ ' + '${0:.0f}$'.format(vout) + ' $\mathrm{km}$ $\mathrm{s^{-1}}$'
            plot_grid_i_small(grid_i, i, vrot, vout, logtau, xmin, ymin, xmax, ymax, vout_lab)
            #grid_i.set_xticks([-200,0,200,400,600], [-200,0,200,400,600])
            i += 1

    plt.savefig('./results/varying_outflow_small.pdf', format='pdf', transparent=False)#, bbox_inches=4)
    plt.close()

# Figure: varying_rotation

In [9]:
logtaus = [5,6,7]
vrots = [0,50,100]
vout = 50

make_grid_plot_voutfixed(logtaus, vrots, vout)

In [10]:
logtaus = [5,6]
vrots = [50,100]
vout = 50

make_grid_plot_voutfixed_small(logtaus, vrots, vout)

# Figure: varying_outflow

In [11]:
logtaus = [5,6,7]
vrot = 50
vouts = [25,50,75]

make_grid_plot_vrotfixed(logtaus, vrot, vouts)

In [12]:
logtaus = [5,6]
vrot = 50
vouts = [25,75]

make_grid_plot_vrotfixed_small(logtaus, vrot, vouts)